In [1]:
!pip install numpy pillow torch torchvision

In [3]:
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

In [7]:
device = torch.device("cpu")

if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_built() and torch.backends.mps.is_available():
    device = torch.device("mps")
print(device)

cpu


In [9]:
# convert image to tensor
# scale the RGB color to be 0-1 and normalize using mean=0.5, std=0.5
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [10]:
train_set = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_set = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_set, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=32,shuffle=False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

  0%|          | 0.00/170M [00:00<?, ?B/s]

100%|██████████| 170M/170M [00:02<00:00, 84.3MB/s] 


In [13]:
image, label = train_set[0]
image.size()

torch.Size([3, 32, 32])

In [20]:
len(train_loader)

1563

In [21]:
class NeuralNet(nn.Module):

    def __init__(self):
        super().__init__()

        ## 3 input channels and 10 output classes
        self.conv1 = nn.Conv2d(3, 12, 5)  # (12, 28, 28) <- 28 = (32-5)/1
        self.pool = nn.MaxPool2d(2, 2)  # (12, 14, 14) <- 28/2
        self.conv2 = nn.Conv2d(12, 24, 5)  # (24, 10, 10) 
        # self.pool = nn.MaxPool2d(2, 2) repeated max pooling # (24, 5, 5) <- 10/2 = 5

        self.fc1 = nn.Linear(24 * 5 * 5, 120) 
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)


    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x))) # conv1 + relu + max pooling
        x = self.pool(F.relu(self.conv2(x))) # conv2 + relu + max pooling
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [23]:
net = NeuralNet()
loss_funcion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [24]:
for epoch in range(10):
    print(f'Training epoch {epoch}...')

    running_loss = 0.0

    for i, data in enumerate(train_loader):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = net(inputs)

        loss = loss_funcion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Loss: {running_loss / len(train_loader):.4f}')

Training epoch 0...


Loss: 2.2572965342039035:.4f
Training epoch 1...
Loss: 1.8129957270637507:.4f
Training epoch 2...
Loss: 1.5616949271942207:.4f
Training epoch 3...
Loss: 1.4260248145230368:.4f
Training epoch 4...
Loss: 1.3307506307072923:.4f
Training epoch 5...
Loss: 1.2521494801274797:.4f
Training epoch 6...
Loss: 1.1843509781230968:.4f
Training epoch 7...
Loss: 1.121929438886014:.4f
Training epoch 8...
Loss: 1.067056511772495:.4f
Training epoch 9...
Loss: 1.0216962743362248:.4f


In [ ]:
torch.save(net.state_dict(), 'trained_net.pth')

In [ ]:
net = NeuralNet()
net.load_state_dict(torch.load('trained_net.pth'))

In [26]:
correct = 0
total = 0

net.eval()

with torch.no_grad():
    for data in test_loader:
        images, labels = data

        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total

print(f'Accuracy: {accuracy}%')

Accuracy: 62.92%
